In [5]:
import random

def fuzz(k):
    return k*random.gauss(0,1)

def dot(v1, v2):
    return sum([(v1i * v2i) for v1i, v2i in zip(v1, v2)])

def make_data(true_params, n=100, k=0.7):
    x = [[random.random() for _ in range(len(true_params)-1)] + [1] for _ in range(n)]
    y = [dot(xi, true_params) + fuzz(k) for xi in x]
    return x, y

def vector_subtract(v1, v2):
    return [(v1i - v2i) for v1i, v2i in zip(v1, v2)]

def scalar_multiply(a, v):
    return [a*vi for vi in v]

def mse(x, y, params):
    yhat = [dot(xi, params) for xi in x]
    return sum([(y_i - yhat_i)**2 for y_i, yhat_i in zip(y, yhat)])/len(y)

def grad_est_i(x, y, params, loss_fn, i, h=0.00001):
    params_nudged = [pi + (h if i == ix else 0) for ix, pi in enumerate(params)]
    return (loss_fn(x, y, params_nudged) - loss_fn(x, y, params)) / h

def grad_est(x, y, params, loss_fn):
    return [grad_est_i(x, y, params, loss_fn, i) for i in range(len(params))]

def sgd(x, y, params_0, loss_fn, grad_fn, lr_0):
    params = params_0
    lr = lr_0
    min_loss = float('inf')
    min_params = None
    iterations_without_improvement = 0
    while iterations_without_improvement < 100:
        loss = loss_fn(x, y, params)
        if loss < min_loss:
            min_loss = loss
            min_params = params
            iterations_without_improvement = 0
        else:
            iterations_without_improvement += 1
            lr *= 0.9
        grad = grad_fn(x, y, params, loss_fn)
        params = vector_subtract(params, scalar_multiply(lr, grad))
    return min_params
    
true_params = [23, 3, -5]
x, y = make_data(true_params)
params_0 = [random.random() for _ in range(len(true_params))]
params = sgd(x, y, params_0, mse, grad_est, 0.1)
print(f"true params: {true_params}")
print(f"initial guess: {params_0}")
print(f"post sgd params: {params}")

true params: [23, 3, -5]
initial guess: [0.20406700017695845, 0.510522174164074, 0.6983853193861417]
post sgd params: [22.750161820931396, 3.034365534950667, -4.94755524812408]
